In [3]:
import constants as c
import functions
import optimizing
import preprocessing
import stationfinding
from visualize import *

import os
from ipywidgets import Button
import asyncio
from ipywidgets import IntSlider, FloatSlider, FloatText, Text, widgets
from IPython.display import display

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [5]:
M_eta_new, P_e_new, P_w_new, data_temp_new = None, None, None, None

def wait_for_change(widget):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.description)
        widget.on_click(getvalue, remove=True)
        # we need to free up the binding to getvalue to avoid an InvalidState error
        # buttons don't support unobserve
        # so use `remove=True`
    widget.on_click(getvalue)
    return future

layout = widgets.Layout(width='auto', height='40px')

room_temp_w = FloatSlider(value=21, min=17, max=27, step = 0.5, description= 'Desired room temperature: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})
return_flow_w = FloatSlider(value=25, min=21, max=50, step = 0.5, description= 'Returning flow temperature: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})
ny_w = FloatSlider(value=0.4, min=0.1, max=1, step = 0.1, description= 'Quality factor of the heat pump: ', continuous_update = False, layout=layout, style= {'description_width': 'initial'})

flow_max_w = FloatText(value= 50, description = 'Maximum flow temperature at minimum temperature: ', layout=layout, style= {'description_width': 'initial'})

out_min_w = FloatText(value= -10, description = 'Minimum outside temperature: ', layout=layout, style= {'description_width': 'initial'})

house_energy_w = FloatText(value= 200, description = 'Energy need in Watt per Kelvin: ', layout=layout, style= {'description_width': 'initial'}) # Watt / Kelvin
storage_power_w = FloatText(value= 200, description = 'Energy need of your storage (DIN...): ', layout=layout, style= {'description_width': 'initial'}) #Watts
storage_volume_w = FloatText(value= 4000, description = 'Storage volume: ', layout=layout, style= {'description_width': 'initial'}) #Liter = kilogram -> to be defined later on

hp_power_el_opt_w = FloatText(value= 5000, description = 'Power of heat pump: ', layout=layout, style= {'description_width': 'initial'}) #Watts -> Power in which heatpump works most efficiently (can be taken into calculation later)
max_heat_cap_w = IntSlider(value=75, description='Maximum heating temperature: ', min=50, max=95, layout=layout, style= {'description_width': 'initial'}) # temperature, up to which the heat pump can maximally heat

address_w = Text(placeholder = 'Examplestreet 1, 12345 Examplecity', description = 'Address: ', layout=layout, style= {'description_width': 'initial'})

button_w = Button(description="GO!")

def main_func():
    global M_eta_new, P_e_new, P_w_new, data_temp_new

    room_temp = room_temp_w.value
    return_flow = return_flow_w.value
    ny = ny_w.value
    house_energy = house_energy_w.value
    storage_power = storage_power_w.value
    storage_volume = storage_volume_w.value
    hp_power_el_opt = hp_power_el_opt_w.value
    max_heat_cap = max_heat_cap_w.value
    address = address_w.value

    flow_max = flow_max_w.value
    out_min = out_min_w.value

    # TODO choose between Forecasting and StationData
    funs = functions.Functions(flow_max=flow_max, room_temp=room_temp, out_min=out_min, return_flow=return_flow, house_energy=house_energy, storage_volume=storage_volume, storage_power=storage_power, ny=ny)
    data_temp = stationfinding.StationFinder(address).main()
    preprocessor = preprocessing.Preprocessor(flow_max=max_heat_cap, room_temp=room_temp, out_min=out_min, return_flow=return_flow, house_energy=house_energy, storage_volume=storage_volume, storage_power=storage_power, ny=ny)
    optimizer = optimizing.Optimizer(functions=funs, max_heat_cap=max_heat_cap, hp_power_el_opt=hp_power_el_opt, return_flow=return_flow, storage_volume=storage_volume)
    
    data_temp = pd.read_table(r'C:\Users\rafaelweinert\PycharmProjects\Heatpump\data\weatherstationdata\produkt_tu_stunde_19480101_20081031_02522.txt', sep=';') #Karlsruhe

    data_temp = data_temp.drop(['eor', 'QN_9', 'RF_TU'], axis=1) # drop end-of-row indicator
    data_temp.rename(columns = {'TT_TU':'temperature', 'MESS_DATUM': 'date', }, inplace = True)
    data_temp.date = pd.to_datetime(data_temp.date.astype(str), format='%Y%m%d%H')
    data_temp[data_temp == -999] = np.nan # rename missing values to numpy nan
    data_temp = data_temp[(data_temp.date >= pd.to_datetime('2007-10-01')) & (data_temp.date <= pd.to_datetime('2007-10-05'))]
    #data_temp = data_temp[(data_temp.date.dt.month >= 10) | (data_temp.date.dt.month <= 3)]
    data_temp.reset_index(drop = True, inplace=True)
    data_temp = preprocessor.add_data(data_temp)
    data_temp = preprocessor.get_local_extrema(data_temp)

    temp_plot = plot_temperature(data_temp, True)
    
    M_eta_new, P_e_new, P_w_new, data_temp_new = optimizer.optimize_periods(data_temp)

    if not os.path.isdir(f"results/{address}"):
        os.makedirs(f'results/{address}')

    data_temp_new.to_csv(rf'.\results\{address}\data_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';', index=False)
    M_eta_new.tofile(rf'.\results\{address}\Eta_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')
    P_e_new.tofile(rf'.\results\{address}\Pe_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')
    P_w_new.tofile(rf'.\results\{address}\Pw_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.csv', sep=';')
    temp_plot.write_html(rf'.\results\{address}\Temp_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html')

    M_eta_hm = plot_heatmap(M_eta_new)
    M_eta_hm.write_html(rf'.\results\{address}\Etahm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

    P_e_hm = plot_heatmap(P_e_new)
    P_e_hm.write_html(rf'.\results\{address}\Pehm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

    P_w_hm = plot_heatmap(P_w_new)
    P_w_hm.write_html(rf'.\results\{address}\Pwhm_ad_{address},vol_{int(storage_volume)},pow_{int(hp_power_el_opt)},en_{int(house_energy)}.html', auto_open=False)

    print(''.center(40, '_'))
    print()
    print('Energy saving potentials: ', "{:10.2f}".format((1 - data_temp_new.P_e_opt.sum() / data_temp_new.energy_needed_elec.sum())*100), '%')
    print()
    print(''.center(40, '_'))
    print()
    print('DONE!')

    return M_eta_new, P_e_new, P_w_new, data_temp_new

async def f():
    x = await wait_for_change(button_w)
    print('------ computing saving potentials -------')
    main_func()

asyncio.create_task(f())

print('Adjust the values of the building and click GO! when you are ready.')

display(room_temp_w)
display(return_flow_w)
display(ny_w)
display(house_energy_w)
display(storage_power_w)
display(storage_volume_w)
display(hp_power_el_opt_w)
display(max_heat_cap_w)
display(address_w)
display(out_min_w)
display(flow_max_w)
display(button_w)


Adjust the values of the building and click GO! when you are ready.


FloatSlider(value=21.0, continuous_update=False, description='Desired room temperature: ', layout=Layout(heigh…

FloatSlider(value=25.0, continuous_update=False, description='Returning flow temperature: ', layout=Layout(hei…

FloatSlider(value=0.4, continuous_update=False, description='Quality factor of the heat pump: ', layout=Layout…

FloatText(value=200.0, description='Energy need in Watt per Kelvin: ', layout=Layout(height='40px', width='aut…

...

IntSlider(value=75, description='Maximum heating temperature: ', layout=Layout(height='40px', width='auto'), m…

Text(value='', description='Address: ', layout=Layout(height='40px', width='auto'), placeholder='Examplestreet…

FloatText(value=-10.0, description='Minimum outside temperature: ', layout=Layout(height='40px', width='auto')…

FloatText(value=50.0, description='Maximum flow temperature at minimum temperature: ', layout=Layout(height='4…

Button(description='GO!', style=ButtonStyle())

In [7]:
storage_volumes = [100, 200, 300, 500, 1000, 3000, 5000, 8000, 12000, 15000, 30000]
for vol in storage_volumes:
    storage_volume_w.value = vol
    address_w.value = 'Karlsruhe'
    M_eta_new, P_e_new, P_w_new, data_temp_new = main_func()
    print('storage volume: ', vol, 'liters')

addresse:  Karlsruhe


period:  3  / 3
________________________________________

Energy saving potentials:      -17.94 %

________________________________________

DONE!
storage volume:  100 liters
addresse:  Karlsruhe


period:  3  / 3
________________________________________

Energy saving potentials:      -10.80 %

________________________________________

DONE!
storage volume:  200 liters
addresse:  Karlsruhe


period:  3  / 3
________________________________________

Energy saving potentials:       -6.94 %

________________________________________

DONE!
storage volume:  300 liters
addresse:  Karlsruhe


KeyboardInterrupt: 